In [9]:
import sqlite3 as sql
import csv
import altair as alt
import json
import plotly.express as px
from collections import Counter
from IPython.display import SVG
import pprint

Requête exécutée sur le serveur : https://dbpedia.org/sparql

In [12]:
csv_file = '/Users/maxime/Documents/dossier_python/data/sparql_birthyear.csv'

In [13]:
csv_content = []
with open(csv_file) as f: 
    rdr = csv.reader(f, delimiter=',')
    for r in rdr:
        csv_content.append(r)
csv_content[:3]

[['o1', 'birthYear'],
 ['http://dbpedia.org/resource/Qusta_ibn_Luqa', '820'],
 ["http://dbpedia.org/resource/Abu_al-Wafa'_Buzjani", '940']]

In [14]:
csv_content[1][0].replace('http://dbpedia.org/resource/','')

'Qusta_ibn_Luqa'

In [15]:
r = []
for i in csv_content[1:]:
    a = i[0].replace('http://dbpedia.org/resource/','')
    b = a.replace('_',' ')
    c = int(i[1])
    r.append([i[0], b, c ])

In [16]:
r[:3]

[['http://dbpedia.org/resource/Qusta_ibn_Luqa', 'Qusta ibn Luqa', 820],
 ["http://dbpedia.org/resource/Abu_al-Wafa'_Buzjani",
  "Abu al-Wafa' Buzjani",
  940],
 ['http://dbpedia.org/resource/Maslama_al-Majriti', 'Maslama al-Majriti', 950]]

In [17]:
len(csv_content)

3583

In [18]:
la = [e[2] for e in r]; la[:3]

[820, 940, 950]

In [19]:
ctr = Counter(la)
ctr_l = list(ctr.items())
ctr_l[:7]

[(820, 1), (940, 1), (950, 1), (953, 1), (970, 1), (980, 1), (1048, 1)]

In [20]:
data = [{'year': e[0], 'eff': e[1]} for e in ctr_l]

In [21]:
with open('/Users/maxime/Documents/dossier_python/data/sparql_birthyear.csv', 'w') as f:
    json.dump(data, f)

In [22]:
alt.Chart.from_dict({
    "data": {
    # "url" : "http://localhost:8888/files/astronomers/notebooks_jupyter/dbpedia_exploration/data/birth_years_grouped.json"
    "values": data
    },
    "mark": "bar", 
    "width": 890,
    "height": 400,
    "encoding": {
        # Remplacer type de x par  : 'quantitative' et observer la différence
        "x": {"field": "year", "type": "ordinal", 
              ## décommenter et observer
              "axis": {"values": [900,1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700 , 1800, 1900, 2000]}
             }, 
         "y": {"field": "eff", "type": "quantitative"},
        

    }
}

)

alt.Chart(...)

In [23]:
ys = [y[2] for y in r]; print (ys[:3], ys[-3:])
min(ys), max(ys)

[820, 940, 950] [1991, 1993, 1993]


(820, 1993)

In [24]:
### Créer une liste d'années complète
y_l = list(range(min(ys), max(ys) + 1, 1))
print(len(y_l))
### Afficher le début et la fin de la série complète
y_l[:4],y_l[-4:]

1174


([820, 821, 822, 823], [1990, 1991, 1992, 1993])

In [25]:
y_r = []
for a in y_l:
    effectif = 0
    noms =  []
    for v in r:
        if a == v[2]:
            effectif += 1
            noms.append(v[1])
            pass
    y_r.append([a, effectif, ', '.join(noms)])  
    
pprint.pprint(y_r[:5])

[[820, 1, 'Qusta ibn Luqa'],
 [821, 0, ''],
 [822, 0, ''],
 [823, 0, ''],
 [824, 0, '']]


In [26]:
[f'Année: {e[0]}, effectif: {e[1]}, personnes: {e[2]}.' if len(e[2]) > 0  
 else f'Année: {e[0]}, effectif: {e[1]}.' for e in y_r][:5]

['Année: 820, effectif: 1, personnes: Qusta ibn Luqa.',
 'Année: 821, effectif: 0.',
 'Année: 822, effectif: 0.',
 'Année: 823, effectif: 0.',
 'Année: 824, effectif: 0.']

In [27]:
data = [{'year': e[0], 'eff': e[1], 'names': e[2]} for e in y_r]

In [28]:
with open('/Users/maxime/Documents/dossier_python/data/sparql_birthyear.csv', 'w') as f:
    json.dump(data, f)

In [29]:
alt.Chart.from_dict({
    "data": {
    ### Choisir si on veut afficher les données de la variable 'data'
    # ou lire le fichier qui se trouve sur le serveur local jupyter (localhost:8888/files)
    "values" : data    
    # "url" : "http://localhost:8888/files/astronomers/notebooks_jupyter/dbpedia_exploration/data/birth_years_grouped_y.json"
    },
    "mark": "bar",
    # {
    #     "type":"bar",
    #     "tooltip": {"content": "data"}
    # }, 
    "encoding": {
        "x": {"field": "year", "type": "ordinal", 
              "axis": {"values": [900,1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700 , 1800, 1900, 2000]}}, # ordinal, quantitative
         "y": {"field": "eff", "type": "quantitative"},
        "tooltip": [
      {"field": "year", "type": "quantitative", "title": "Year"},
     {"field": "names", "type": "nominal", "title": "Names"}
    ]
    },
    "width": 890, 
    "height": 300
}

)

alt.Chart(...)

In [30]:
min(y_l), max(y_l)

(820, 1993)

In [31]:
periode = 20
per_l = list(range(800, 2000, periode))
per_ll = [(l, l+ periode -1) for l in per_l]
len(per_ll),per_ll[:3],per_ll[-3:]

(60,
 [(800, 819), (820, 839), (840, 859)],
 [(1940, 1959), (1960, 1979), (1980, 1999)])

In [32]:
per_r = []
for a in per_ll:
    if a[1] < 1000:
        label = f'0{a[0]}_0{a[1]}'
    else:
        label = f'{a[0]}_{a[1]}'
    # réinitialiser ces variables à chaque boucle
    effectif = 0
    noms =  []
    for v in r:
        if v[2] >= a[0] and v[2] <= a[1]:
            effectif += 1
            noms.append(v[1])
            # pass
    per_r.append([label, effectif, noms])

per_r[:11]        


[['0800_0819', 0, []],
 ['0820_0839', 1, ['Qusta ibn Luqa']],
 ['0840_0859', 0, []],
 ['0860_0879', 0, []],
 ['0880_0899', 0, []],
 ['0900_0919', 0, []],
 ['0920_0939', 0, []],
 ['0940_0959', 3, ["Abu al-Wafa' Buzjani", 'Maslama al-Majriti', 'Al-Karaji']],
 ['0960_0979', 1, ['Abu Nasr Mansur']],
 ['0980_0999', 1, ['Abu Mansur al-Baghdadi']],
 ['1000_1019', 0, []]]

In [33]:
labels = ['<br>'.join([m.strip() for m in i[2]]) for i in per_r] ; len(labels),labels[0]

(60, '')

In [34]:
data = [{'per': e[0], 'eff': e[1], 'names': '\n'.join(e[2])} for e in per_r]

In [35]:
with open('/Users/maxime/Documents/dossier_python/data/sparql_birthyear.csv', 'w') as f:
    json.dump(data, f)

In [36]:
alt.Chart.from_dict({
    "data": {
    "values" : data    
    # "url" : "http://localhost:8888/files/astronomers/notebooks_jupyter/dbpedia_exploration/data/birth_years_grouped_per.json"
    },
    "mark": "bar",
    # {
    #     "type":"bar",
    #     "tooltip": {"content": "data"}
    # }, 
    "encoding": {
        "x": {"field": "per", "type": "ordinal", "title": "Period" 
             }, 
         "y": {"field": "eff", "type": "quantitative", "title": "Effectif"},
        "tooltip": [
      {"field": "eff", "type": "nominal", "title": "Effectif"},
     {"field": "names", "type": "nominal", "title": "Names"},
    ]
    },
  "config": {

    "axisX": {
      "labelAngle": 60
    }
  },
    "width": 890, 
    "height": 600
}
)

alt.Chart(...)

In [37]:
data = [{'per': e[0], 'eff': e[1], 'names': '\n'.join(e[2])} for e in per_r]

In [38]:
### Ajouter une moyenne glissante
# Documentation:
#     https://joelostblom.github.io/altair-docs/gallery/bar_with_rolling_mean.html

## https://altair-viz.github.io/user_guide/data.html
data_trsf = alt.Data(values=data)

bar = alt.Chart(data_trsf).mark_bar().encode(
    x='per:N',
    y='eff:Q',
    tooltip=('eff:Q', 'names:N')
   
)

line = alt.Chart(data_trsf).mark_line(color='red').transform_window(
    # The field to average
    rolling_mean='mean(eff)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    x='per:O',
    y='rolling_mean:Q'
)

(bar + line).properties(height= 600, width=890,  title = 'Distribution des naissances par périodes de vingt ans')

alt.LayerChart(...)